In [9]:
run location_recommender_with_friends_network.ipynb

Nodes =  145040
Edges =  1832987
[420315  21714   9241 ...  17144  13791  13776]


interactive(children=(SelectionSlider(continuous_update=False, description='UserId:', index=2, options=(1, 2, …

In [10]:
import random
# evaluation: choose 30% of values as test data
samples = [(i,j,v) for i,j,v in zip(user_item_sparse.row, user_item_sparse.col, user_item_sparse.data)]
random.shuffle(samples)

In [29]:
N = (len(samples)*70)//100
train = samples[0:N]
test = samples[N:]

user_item_train = sparse.coo_matrix(([v for _,_,v in train], ([i for i,_,_ in train],[j for _,j,_ in train])),shape=user_item_sparse.shape)
user_item_test = sparse.coo_matrix(([v for _,_,v in test], ([i for i,_,_ in test],[j for _,j,_ in test])),shape=user_item_sparse.shape)

# store the original user_item_sparse
user_item_orig_csr = user_item_sparse.tocsr()

# use training data for all predictions
# user_item_csr is used by the prediction functions
user_item_csr = user_item_train.tocsr()

In [30]:
# for precision and recall: 
# prediction functions use user_item_csr to predict which now has only 70% of data
# we predict locations and see how many can be recovered by comparing with original set
# Precision@N defines the ratio of recovered POI to the N recommended POI

relevant_usern = set()
for s in test:
    if s[0] not in relevant_usern:
        relevant_usern.add(s[0])
        
def precision_at_k(K=10,f=1):
    p = 0.0
    user_p = list(relevant_usern)
    random.shuffle(user_p)
    for un in user_p[0:500]:
        if f == 1:
            retrieved = recommend_locations_with_friends(user_list[un],K)
        else:
            retrieved = recommend_locations_without_friends(user_list[un],K)
            
        # how many of these are relevant?
        relevant = 0
        for loc in retrieved:
            locn = loc_idx[loc]
            if user_item_orig_csr[un,locn] != 0:
                relevant += 1
        
        p += (relevant*1.0)/len(retrieved)
    
    return (p/len(user_p[0:500]))

print("precision@10, without friends network : ",precision_at_k(10,0))
print("precision@10, with friends network : ",precision_at_k(10,1))
print("precision@5, without friends network : ",precision_at_k(5,0))
print("precision@5, with friends network : ",precision_at_k(5,1))


precision@10, without friends network :  0.009600000000000001
precision@10, with friends network :  0.03498665310640214
precision@5, without friends network :  0.020800000000000006
precision@5, with friends network :  0.037202065581595556
